In [1]:
import sys
from pathlib import Path
from os import getcwd

from datetime import date, timedelta

sys.path.append(str(Path(getcwd()).parent))
print(sys.path)

['/home/vinnylg/projects/covid19datascience/notebooks', '/home/vinnylg/anaconda3/lib/python38.zip', '/home/vinnylg/anaconda3/lib/python3.8', '/home/vinnylg/anaconda3/lib/python3.8/lib-dynload', '', '/home/vinnylg/.local/lib/python3.8/site-packages', '/home/vinnylg/anaconda3/lib/python3.8/site-packages', '/home/vinnylg/.local/lib/python3.8/site-packages/IPython/extensions', '/home/vinnylg/.ipython', '/home/vinnylg/projects/covid19datascience']


In [2]:
from os.path import join
import pandas as pd
pd.set_option('display.max_columns', None)
from sys import exit

In [3]:
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.tb_pacientes import TbPacientes
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash
from bulletin.utils.static import Municipios
from bulletin.utils.xls_ import fit_cols
from bulletin import default_input


---------------------------------------------------------------

root: /home/vinnylg/projects/covid19datascience/bulletin
parent: /home/vinnylg/projects/covid19datascience
default_input: /home/vinnylg/projects/covid19datascience/input
default_output: /home/vinnylg/projects/covid19datascience/output

---------------------------------------------------------------



In [4]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)

In [5]:
tb_pacientes = TbPacientes()
tb_pacientes.update()
casost = tb_pacientes.get()
# casost = pd.read_pickle('tb_pacientes.pkl')#tb_pacientes.get()
casost['nome'] = casost['nome'].apply(lambda nome: ''.join(filter(lambda x: ('A' <= x <= 'Z') or ( x == ' '), nome)))
casost = casost.sort_values(['dt_notificacao','nome','idade'])
print(f"casost len: {len(casost)}\n")
casost.loc[casost['obito']=='Sim','obito'] = 'SIM'
casost.loc[casost['obito']!='SIM','obito'] = 'NAO'
print(f"casost len: {len(casost)}\n")
casost.groupby('obito')[['sexo']].count()

Reading TbPacientes time elepsed:00:10:24
casost len: 1181439

casost len: 1181439



,sexo
obito,
NAO,1151765
SIM,29674


In [6]:
casost

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais,idade_menos,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag,hash_obito
8951,8838,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,CIANORTE,CIANORTE,LACEN,2020-03-12,2020-03-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,2020-04-03 00:00:00,Notificações,Recuperado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,47,NaT,NaT,48,46,ELIANACRISTINACUNHADEOLIVEIRASOUZA47CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA46CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA48CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA47CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA46CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA48CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA12032020,NaN
11016,10932,410690,410690,FLAMARION PETRIV DE ABREU,M,43,CURITIBA,CURITIBA,LACEN,2020-03-12,2020-03-12,2020-03-07 00:00:00,NaN,43897,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,43,NaT,NaT,44,42,FLAMARIONPETRIVDEABREU43CURITIBA,FLAMARIONPETRIVDEABREU42CURITIBA,FLAMARIONPETRIVDEABREU44CURITIBA,FLAMARIONPETRIVDEABREU43CURITIBA,FLAMARIONPETRIVDEABREU42CURITIBA,FLAMARIONPETRIVDEABREU44CURITIBA,FLAMARIONPETRIVDEABREU12032020,NaN
11017,10933,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,CURITIBA,CURITIBA,LACEN,2020-03-12,2020-03-12,2020-03-08 00:00:00,NaN,43897,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,15,NaT,NaT,16,14,FLAMARIONPETRIVDEABREUJUNIOR15CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR14CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR16CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR15CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR14CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR16CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR12032020,NaN
11550,11470,410690,410690,GABRIELA FUGANTI CASAGRANDE,F,26,CURITIBA,CURITIBA,Genoprimer,2020-03-16,2020-03-12,2020-03-11 00:00:00,NaN,07/03/2020 a 10/03/2020,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,26,NaT,NaT,27,25,GABRIELAFUGANTICASAGRANDE26CURITIBA,GABRIELAFUGANTICASAGRANDE25CURITIBA,GABRIELAFUGANTICASAGRANDE27CURITIBA,GABRIELAFUGANTICASAGRANDE26CURITIBA,GABRIELAFUGANTICASAGRANDE25CURITIBA,GABRIELAFUGANTICASAGRANDE27CURITIBA,GABRIELAFUGANTICASAGRANDE16032020,NaN
11572,11492,410690,410690,GABRIELA SCHNEIDER GUGELMIN,F,26,CURITIBA,CURITIBA,Genoprimer,2020-03-18,2020-03-12,2020-03-15 00:00:00,NaN,08/03/2020 a 10/03/2020,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,26,NaT,NaT,27,25,GABRIELASCHNEIDERGUGELMIN26CURITIBA,GABRIELASCHNEIDERGUGELMIN25CURITIBA,GABRIELASCHNEIDERGUGELMIN27CURITIBA,GABRIELASCHNEIDERGUGELMIN26CURITIBA,GABRIELASCHNEIDERGUGELMIN25CURITIBA,GABRIELASCHNEIDERGUGELMIN27CURITIBA,GABRIELASCHNEIDERGUGELMIN18032020,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8476,1262903,411990,411990,ZORAIDE SIMAO,F,62,PONTA GROSSA,PONTA GROSSA,"COVID-19, Biologia Molecular",2021-05-07,2021-06-18,2021-04-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIM,2021-05-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,130.0,62,2021-06-18,NaT,63,61,ZORAIDESIMAO62PONTAGROSSA,ZORAIDESIMAO61PONTAGROSSA,ZORAIDESIMAO63PONTAGROSSA,ZORAIDESIMAO62PONTAGROSSA,ZORAIDESIMAO61PONTAGROSSA,ZORAIDESIMAO63PONTAGROSSA,ZORAIDESIMAO07052021,ZORAIDESIMAO26052021
678217,1272836,410520,410520,ZORLEI DOS SANTOS HARPS,M,38,CERRO AZUL,CERRO AZUL,"COVID-19,

In [7]:
casost.groupby('status')[['sexo']].count()

,sexo
status,
Recuperado,830854
recuperado,2


In [8]:
casosct = casost.copy()

casosct['evolucao'] = 3
casosct.loc[casosct['obito']=='SIM','evolucao'] = 2
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'evolucao'] = 1

In [9]:
casosct.loc[casosct['obito']=='SIM','data_evolucao'] = casosct.loc[casosct['obito']=='SIM','dt_obito'] 
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_evolucao'] = casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'dt_atualizacao']

In [10]:
casosct.loc[casosct['obito']=='SIM','data_com_evolucao'] = casosct.loc[casosct['obito']=='SIM','dt_com_obito'] 
casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'data_com_evolucao'] = casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'dt_obito']
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_com_evolucao'] = casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'dt_com_recuperado'] 

In [11]:
casosct.groupby('evolucao')[['sexo']].count()

,sexo
evolucao,
1,830856
2,29674
3,320909


In [12]:
casosct['uf_resid'] = 'PR'
casosct['ibge_res_pr'] = casosct['ibge_res_pr'].apply(int)

casosct.loc[casosct['ibge_res_pr']==999999,'uf_resid'] = casosct.loc[casosct['ibge_res_pr']==999999,'mun_resid'].apply(lambda x: x.split('/')[-1])
casosct.loc[casosct['ibge_res_pr']==999999,'mun_resid'] = casosct.loc[casosct['ibge_res_pr']==999999,'mun_resid'].apply(lambda x: x.split('/')[0])

casos_fora = pd.merge(casosct.loc[casosct['ibge_res_pr']==999999].drop(columns=['ibge_res_pr']),municipios[['ibge','mun_resid','uf']].rename(columns={'uf':'uf_resid','ibge':'ibge_res_pr'}), on=['mun_resid','uf_resid'], how='left')
casos_fora.loc[casos_fora['ibge_res_pr'].isna(),'ibge_res_pr'] = 999999
casos_fora['ibge_res_pr'] = casos_fora['ibge_res_pr'].astype(int)
casos_fora = casos_fora.set_index('identificacao')
casosct = casosct.set_index('identificacao')
casosct.update(casos_fora)

casosct = casosct.reset_index()

casosct['ibge_res_pr'] = casosct['ibge_res_pr'].astype(int)
casosct['idade'] = casosct['idade'].astype(int)

casosct.loc[casosct['uf_resid']!='PR']

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais,idade_menos,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag,hash_obito,data_evolucao,data_com_evolucao,uf_resid
5,13673,355030,410690,ITALO FERNANDO TROMBINI FILHO,M,28,SAO PAULO,CURITIBA,Genoprimer,2020-03-16,2020-03-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,2020-06-17 00:00:00,SMS,Recuperado,NaN,NaN,NaN,NaN,1.0,NaN,NaN,14.0,28.0,NaT,2020-08-05,29.0,27.0,ITALOFERNANDOTROMBINIFILHO28SAOPAULOSP,ITALOFERNANDOTROMBINIFILHO27SAOPAULOSP,ITALOFERNANDOTROMBINIFILHO29SAOPAULOSP,ITALOFERNANDOTROMBINIFILHO28CURITIBA,ITALOFERNANDOTROMBINIFILHO27CURITIBA,ITALOFERNANDOTROMBINIFILHO29CURITIBA,ITALOFERNANDOTROMBINIFILHO16032020,NaN,2020-06-17 00:00:00,2020-08-05,SP
63,19531,530010,410690,LUCIANO PUCHALSKI,M,59,BRASILIA,CURITIBA,LACEN,2020-03-23,2020-03-24,2020-03-13 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,23.0,59.0,NaT,NaT,60.0,58.0,LUCIANOPUCHALSKI59BRASILIADF,LUCIANOPUCHALSKI58BRASILIADF,LUCIANOPUCHALSKI60BRASILIADF,LUCIANOPUCHALSKI59CURITIBA,LUCIANOPUCHALSKI58CURITIBA,LUCIANOPUCHALSKI60CURITIBA,LUCIANOPUCHALSKI23032020,NaN,NaT,NaT,DF
77,15204,355030,410370,JOAO HENRIQUE FONZAR BEGNINI,M,37,SAO PAULO,CAMBE,LACEN,2020-03-25,2020-03-25,2020-03-17 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,23.0,37.0,NaT,NaT,38.0,36.0,JOAOHENRIQUEFONZARBEGNINI37SAOPAULOSP,JOAOHENRIQUEFONZARBEGNINI36SAOPAULOSP,JOAOHENRIQUEFONZARBEGNINI38SAOPAULOSP,JOAOHENRIQUEFONZARBEGNINI37CAMBE,JOAOHENRIQUEFONZARBEGNINI36CAMBE,JOAOHENRIQUEFONZARBEGNINI38CAMBE,JOAOHENRIQUEFONZARBEGNINI25032020,NaN,NaT,NaT,SP
175,19629,330330,411435,LUCINEIA ALVES DE ANHAIA,F,38,NITEROI,MANFRINOPOLIS,LACEN,2020-03-31,2020-03-31,2020-03-22 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,23.0,38.0,NaT,NaT,39.0,37.0,LUCINEIAALVESDEANHAIA38NITEROIRJ,LUCINEIAALVESDEANHAIA37NITEROIRJ,LUCINEIAALVESDEANHAIA39NITEROIRJ,LUCINEIAALVESDEANHAIA38MANFRINOPOLIS,LUCINEIAALVESDEANHAIA37MANFRINOPOLIS,LUCINEIAALVESDEANHAIA39MANFRINOPOLIS,LUCINEIAALVESDEANHAIA31032020,NaN,NaT,NaT,RJ
218,25622,355030,410690,PAULO FRANCISCO NETO,M,25,SAO PAULO,CURITIBA,LACEN,2020-03-31,2020-04-01,2020-03-12 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,2020-03-19 00:00:00,SMS,Recuperado,NaN,NaN,NaN,NaN,1.0,NaN,NaN,23.0,25.0,NaT,2020-08-05,26.0,24.0,PAULOFRANCISCONETO25SAOPAULOSP,PAULOFRANCISCONETO24SAOPAULOSP,PAULOFRANCISCONETO26SAOPAULOSP,PAULOFRANCISCONETO25CURITIBA,PAULOFRANCISCONETO24CURITIBA,PAULOFRANCISCONETO26CURITIBA,PAULOFRANCISCONETO31032020,NaN,2020-03-19 00:00:00,2020-08-05,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181159,1263711,355030,411990,VICENTE LUIZ BARONI PREDO,M,57,SAO PAULO,PONTA GROSSA,Teste Rápido Antígeno (Imunocromatografia),2021-05-25,2021-06-18,2021-05-20 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,192.0,57.0,NaT,NaT,58.0,56.0,VICENTELUIZBARONIPREDO57SAOPAULOSP,VICENTELUIZBARONIPREDO56SAOPAULOSP,VICENTELUIZBARONIPREDO58SAOPAULOSP,VICENTELUIZBARONIPREDO57PONTAGROSSA,VICENTELUIZBARONIPREDO56PONTAGROSSA,VICENTELUIZBARONIPREDO58PONTAGROSSA,VICENTELUIZBARONIPREDO25052021,NaN,NaT,NaT,SP
1181268,1263592,210020,410180,WAGNER COSTA SILVA,M,43,ALCANTARA,ARAUCARIA,Teste R

In [13]:
casosct.groupby(['evolucao'])[['sexo']].count()

,sexo
evolucao,
1.0,830856
2.0,29674
3.0,320909


In [14]:
casosct['id_notifica'] = -1
casosct['ibge_resid'] = casosct['ibge_res_pr']
casosct['ibge_atend'] = casosct['ibge_atend_pr'].apply(int)
casosct['comunicacao'] = casosct['dt_notificacao']
casosct['is'] = casosct['dt_inicio_sintomas']

casosct = casosct[['identificacao', 'id_notifica', 'uf_resid', 'ibge_resid', 'ibge_atend', 'nome', 'sexo', 'idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao', 'data_evolucao', 'data_com_evolucao']]
casosct

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao
0,8838,-1,PR,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,LACEN,2020-03-12,2020-03-12,NaN,1.0,2020-04-03 00:00:00,NaT
1,10932,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU,M,43,LACEN,2020-03-12,2020-03-12,2020-03-07 00:00:00,1.0,NaN,NaT
2,10933,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,LACEN,2020-03-12,2020-03-12,2020-03-08 00:00:00,1.0,NaN,NaT
3,11470,-1,PR,410690,410690,GABRIELA FUGANTI CASAGRANDE,F,26,Genoprimer,2020-03-16,2020-03-12,2020-03-11 00:00:00,1.0,NaN,NaT
4,11492,-1,PR,410690,410690,GABRIELA SCHNEIDER GUGELMIN,F,26,Genoprimer,2020-03-18,2020-03-12,2020-03-15 00:00:00,1.0,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181434,1262903,-1,PR,411990,411990,ZORAIDE SIMAO,F,62,"COVID-19, Biologia Molecular",2021-05-07,2021-06-18,2021-04-30 00:00:00,2.0,2021-05-26 00:00:00,2021-06-18
1181435,1272836,-1,PR,410520,410520,ZORLEI DOS SANTOS HARPS,M,38,"COVID-19, Biologia Molecular",2021-06-10,2021-06-18,2021-06-08 00:00:00,3.0,NaT,NaT
1181436,1270234,-1,PR,411695,411695,ZULMIRA APARECIDA BASSANEZI CHIELE,F,48,"COVID-19, Biologia Molecular",2021-06-17,2021-06-18,2021-06-12 00:00:00,3.0,NaT,NaT
1181437,1272700,-1,PR,412625,411520,ZULMIRA RIBEIRO MARTINS,F,46,Teste Rápido Antígeno (Imunocromatografia),2021-06-17,2021-06-18,2021-06-11 00:00:00,3.0,NaT,NaT


In [15]:
casos_confirmados = casosct.copy()
casos_confirmados

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao
0,8838,-1,PR,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,LACEN,2020-03-12,2020-03-12,NaN,1.0,2020-04-03 00:00:00,NaT
1,10932,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU,M,43,LACEN,2020-03-12,2020-03-12,2020-03-07 00:00:00,1.0,NaN,NaT
2,10933,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,LACEN,2020-03-12,2020-03-12,2020-03-08 00:00:00,1.0,NaN,NaT
3,11470,-1,PR,410690,410690,GABRIELA FUGANTI CASAGRANDE,F,26,Genoprimer,2020-03-16,2020-03-12,2020-03-11 00:00:00,1.0,NaN,NaT
4,11492,-1,PR,410690,410690,GABRIELA SCHNEIDER GUGELMIN,F,26,Genoprimer,2020-03-18,2020-03-12,2020-03-15 00:00:00,1.0,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181434,1262903,-1,PR,411990,411990,ZORAIDE SIMAO,F,62,"COVID-19, Biologia Molecular",2021-05-07,2021-06-18,2021-04-30 00:00:00,2.0,2021-05-26 00:00:00,2021-06-18
1181435,1272836,-1,PR,410520,410520,ZORLEI DOS SANTOS HARPS,M,38,"COVID-19, Biologia Molecular",2021-06-10,2021-06-18,2021-06-08 00:00:00,3.0,NaT,NaT
1181436,1270234,-1,PR,411695,411695,ZULMIRA APARECIDA BASSANEZI CHIELE,F,48,"COVID-19, Biologia Molecular",2021-06-17,2021-06-18,2021-06-12 00:00:00,3.0,NaT,NaT
1181437,1272700,-1,PR,412625,411520,ZULMIRA RIBEIRO MARTINS,F,46,Teste Rápido Antígeno (Imunocromatografia),2021-06-17,2021-06-18,2021-06-11 00:00:00,3.0,NaT,NaT


In [16]:
casos_confirmados.dtypes

identificacao                 int64
id_notifica                   int64
uf_resid                     object
ibge_resid                    int64
ibge_atend                    int64
nome                         object
sexo                         object
idade                         int64
laboratorio                  object
dt_diag              datetime64[ns]
comunicacao          datetime64[ns]
is                           object
evolucao                    float64
data_evolucao                object
data_com_evolucao    datetime64[ns]
dtype: object

In [17]:
casos_confirmados.loc[casos_confirmados['is'].notna(),'is'] = casos_confirmados.loc[casos_confirmados['is'].notna(),'is'].apply(str).str[:10]
casos_confirmados['is'] = pd.to_datetime(casos_confirmados['is'],format='%d/%m/%Y',errors='ignore')
casos_confirmados['is'] = pd.to_datetime(casos_confirmados['is'],format='%Y-%m-%d',errors='coerce')
casos_confirmados['is']

0                NaT
1         2020-03-07
2         2020-03-08
3         2020-03-11
4         2020-03-15
             ...    
1181434   2021-04-30
1181435   2021-06-08
1181436   2021-06-12
1181437   2021-06-11
1181438   2021-06-07
Name: is, Length: 1181439, dtype: datetime64[ns]

In [18]:
casos_confirmados.loc[casos_confirmados['data_evolucao'].notna(),'data_evolucao'] = casos_confirmados.loc[casos_confirmados['data_evolucao'].notna(),'data_evolucao'].apply(str).str[:10]
casos_confirmados['data_evolucao'] = pd.to_datetime(casos_confirmados['data_evolucao'],format='%d/%m/%Y',errors='ignore')
casos_confirmados['data_evolucao'] = pd.to_datetime(casos_confirmados['data_evolucao'],format='%Y-%m-%d',errors='coerce')
casos_confirmados['data_evolucao']

0         2020-04-03
1                NaT
2                NaT
3                NaT
4                NaT
             ...    
1181434   2021-05-26
1181435          NaT
1181436          NaT
1181437          NaT
1181438          NaT
Name: data_evolucao, Length: 1181439, dtype: datetime64[ns]

In [19]:
casos_confirmados['evolucao'] = casos_confirmados['evolucao'].astype(int)
casos_confirmados.dtypes

identificacao                 int64
id_notifica                   int64
uf_resid                     object
ibge_resid                    int64
ibge_atend                    int64
nome                         object
sexo                         object
idade                         int64
laboratorio                  object
dt_diag              datetime64[ns]
comunicacao          datetime64[ns]
is                   datetime64[ns]
evolucao                      int64
data_evolucao        datetime64[ns]
data_com_evolucao    datetime64[ns]
dtype: object

In [22]:
casos_confirmados['hash'] = ( casos_confirmados['nome'].apply(normalize_hash) +
               casos_confirmados['idade'].astype(int).astype(str) +
               casos_confirmados['ibge_resid'].astype(int).astype(str) )

casos_confirmados['hash_less'] = ( casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['idade'].apply(lambda x: str(int(x-1))) +
                    casos_confirmados['ibge_resid'].astype(int).astype(str) )

casos_confirmados['hash_more'] = ( casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['idade'].apply(lambda x: str(int(x+1))) +
                    casos_confirmados['ibge_resid'].astype(int).astype(str) )

casos_confirmados['hash_atend'] = (casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['idade'].astype(int).astype(str) +
                    casos_confirmados['ibge_atend'].astype(int).astype(str) )

casos_confirmados['hash_less_atend'] = (casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['idade'].apply(lambda x: str(int(x-1))) +
                    casos_confirmados['ibge_atend'].astype(int).astype(str) )

casos_confirmados['hash_more_atend'] = (casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['idade'].apply(lambda x: str(int(x+1))) +
                    casos_confirmados['ibge_atend'].astype(int).astype(str) )

casos_confirmados['hash_diag'] = ( casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['dt_diag'].apply(date_hash) )

In [23]:
casos_confirmados = casos_confirmados[['identificacao','id_notifica','uf_resid','ibge_resid','ibge_atend','nome','sexo','idade','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','hash','hash_less','hash_more','hash_atend','hash_less_atend','hash_more_atend','hash_diag']]

In [24]:
casos_confirmados.loc[casos_confirmados['uf_resid']!='PR']

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag
5,13673,-1,SP,355030,410690,ITALO FERNANDO TROMBINI FILHO,M,28,Genoprimer,2020-03-16,2020-03-12,NaT,1,2020-06-17,2020-08-05,ITALOFERNANDOTROMBINIFILHO28355030,ITALOFERNANDOTROMBINIFILHO27355030,ITALOFERNANDOTROMBINIFILHO29355030,ITALOFERNANDOTROMBINIFILHO28410690,ITALOFERNANDOTROMBINIFILHO27410690,ITALOFERNANDOTROMBINIFILHO29410690,ITALOFERNANDOTROMBINIFILHO16032020
63,19531,-1,DF,530010,410690,LUCIANO PUCHALSKI,M,59,LACEN,2020-03-23,2020-03-24,2020-03-13,3,NaT,NaT,LUCIANOPUCHALSKI59530010,LUCIANOPUCHALSKI58530010,LUCIANOPUCHALSKI60530010,LUCIANOPUCHALSKI59410690,LUCIANOPUCHALSKI58410690,LUCIANOPUCHALSKI60410690,LUCIANOPUCHALSKI23032020
77,15204,-1,SP,355030,410370,JOAO HENRIQUE FONZAR BEGNINI,M,37,LACEN,2020-03-25,2020-03-25,2020-03-17,3,NaT,NaT,JOAOHENRIQUEFONZARBEGNINI37355030,JOAOHENRIQUEFONZARBEGNINI36355030,JOAOHENRIQUEFONZARBEGNINI38355030,JOAOHENRIQUEFONZARBEGNINI37410370,JOAOHENRIQUEFONZARBEGNINI36410370,JOAOHENRIQUEFONZARBEGNINI38410370,JOAOHENRIQUEFONZARBEGNINI25032020
175,19629,-1,RJ,330330,411435,LUCINEIA ALVES DE ANHAIA,F,38,LACEN,2020-03-31,2020-03-31,2020-03-22,3,NaT,NaT,LUCINEIAALVESDEANHAIA38330330,LUCINEIAALVESDEANHAIA37330330,LUCINEIAALVESDEANHAIA39330330,LUCINEIAALVESDEANHAIA38411435,LUCINEIAALVESDEANHAIA37411435,LUCINEIAALVESDEANHAIA39411435,LUCINEIAALVESDEANHAIA31032020
218,25622,-1,SP,355030,410690,PAULO FRANCISCO NETO,M,25,LACEN,2020-03-31,2020-04-01,2020-03-12,1,2020-03-19,2020-08-05,PAULOFRANCISCONETO25355030,PAULOFRANCISCONETO24355030,PAULOFRANCISCONETO26355030,PAULOFRANCISCONETO25410690,PAULOFRANCISCONETO24410690,PAULOFRANCISCONETO26410690,PAULOFRANCISCONETO31032020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181159,1263711,-1,SP,355030,411990,VICENTE LUIZ BARONI PREDO,M,57,Teste Rápido Antígeno (Imunocromatografia),2021-05-25,2021-06-18,2021-05-20,3,NaT,NaT,VICENTELUIZBARONIPREDO57355030,VICENTELUIZBARONIPREDO56355030,VICENTELUIZBARONIPREDO58355030,VICENTELUIZBARONIPREDO57411990,VICENTELUIZBARONIPREDO56411990,VICENTELUIZBARONIPREDO58411990,VICENTELUIZBARONIPREDO25052021
1181268,1263592,-1,MA,210020,410180,WAGNER COSTA SILVA,M,43,Teste Rápido IgM/IgG não discriminatório (Imun...,2021-05-13,2021-06-18,NaT,3,NaT,NaT,WAGNERCOSTASILVA43210020,WAGNERCOSTASILVA42210020,WAGNERCOSTASILVA44210020,WAGNERCOSTASILVA43410180,WAGNERCOSTASILVA42410180,WAGNERCOSTASILVA44410180,WAGNERCOSTASILVA13052021
1181320,1264738,-1,GO,520110,411520,WELLINGTON DE FREITAS,M,41,Teste Rápido IgM/IgG não discriminatório (Imun...,2021-06-09,2021-06-18,NaT,3,NaT,NaT,WELLINGTONDEFREITAS41520110,WELLINGTONDEFREITAS40520110,WELLINGTONDEFREITAS42520110,WELLINGTONDEFREITAS41411520,WELLINGTONDEFREITAS40411520,WELLINGTONDEFREITAS42411520,WELLINGTONDEFREITAS09062021
1181392,1271562,-1,MS,500270,411520,WOLNEY MARQUES DE SOUZA,M,44,Teste Rápido Antígeno (Imunocromatografia),2021-06-11,2021-06-18,2021-06-05,3,NaT,NaT,WOLNEYMARQUESDESOUZA44500270,WOLNEYMARQUESDESOUZA43500270,WOLNEYMARQUESDESOUZA45500270,WOLNEYMARQUESDESOUZA44411520,WOLNEYMARQUESDESOUZA43411520,WOLNEYMARQUESDESOUZA45411520,WOLNEYMARQUESDESOUZA11062021


In [25]:
casos_confirmados

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag
0,8838,-1,PR,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,LACEN,2020-03-12,2020-03-12,NaT,1,2020-04-03,NaT,ELIANACRISTINACUNHADEOLIVEIRASOUZA47410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA46410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA48410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA47410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA46410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA48410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA12032020
1,10932,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU,M,43,LACEN,2020-03-12,2020-03-12,2020-03-07,1,NaT,NaT,FLAMARIONPETRIVDEABREU43410690,FLAMARIONPETRIVDEABREU42410690,FLAMARIONPETRIVDEABREU44410690,FLAMARIONPETRIVDEABREU43410690,FLAMARIONPETRIVDEABREU42410690,FLAMARIONPETRIVDEABREU44410690,FLAMARIONPETRIVDEABREU12032020
2,10933,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,LACEN,2020-03-12,2020-03-12,2020-03-08,1,NaT,NaT,FLAMARIONPETRIVDEABREUJUNIOR15410690,FLAMARIONPETRIVDEABREUJUNIOR14410690,FLAMARIONPETRIVDEABREUJUNIOR16410690,FLAMARIONPETRIVDEABREUJUNIOR15410690,FLAMARIONPETRIVDEABREUJUNIOR14410690,FLAMARIONPETRIVDEABREUJUNIOR16410690,FLAMARIONPETRIVDEABREUJUNIOR12032020
3,11470,-1,PR,410690,410690,GABRIELA FUGANTI CASAGRANDE,F,26,Genoprimer,2020-03-16,2020-03-12,2020-03-11,1,NaT,NaT,GABRIELAFUGANTICASAGRANDE26410690,GABRIELAFUGANTICASAGRANDE25410690,GABRIELAFUGANTICASAGRANDE27410690,GABRIELAFUGANTICASAGRANDE26410690,GABRIELAFUGANTICASAGRANDE25410690,GABRIELAFUGANTICASAGRANDE27410690,GABRIELAFUGANTICASAGRANDE16032020
4,11492,-1,PR,410690,410690,GABRIELA SCHNEIDER GUGELMIN,F,26,Genoprimer,2020-03-18,2020-03-12,2020-03-15,1,NaT,NaT,GABRIELASCHNEIDERGUGELMIN26410690,GABRIELASCHNEIDERGUGELMIN25410690,GABRIELASCHNEIDERGUGELMIN27410690,GABRIELASCHNEIDERGUGELMIN26410690,GABRIELASCHNEIDERGUGELMIN25410690,GABRIELASCHNEIDERGUGELMIN27410690,GABRIELASCHNEIDERGUGELMIN18032020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181434,1262903,-1,PR,411990,411990,ZORAIDE SIMAO,F,62,"COVID-19, Biologia Molecular",2021-05-07,2021-06-18,2021-04-30,2,2021-05-26,2021-06-18,ZORAIDESIMAO62411990,ZORAIDESIMAO61411990,ZORAIDESIMAO63411990,ZORAIDESIMAO62411990,ZORAIDESIMAO61411990,ZORAIDESIMAO63411990,ZORAIDESIMAO07052021
1181435,1272836,-1,PR,410520,410520,ZORLEI DOS SANTOS HARPS,M,38,"COVID-19, Biologia Molecular",2021-06-10,2021-06-18,2021-06-08,3,NaT,NaT,ZORLEIDOSSANTOSHARPS38410520,ZORLEIDOSSANTOSHARPS37410520,ZORLEIDOSSANTOSHARPS39410520,ZORLEIDOSSANTOSHARPS38410520,ZORLEIDOSSANTOSHARPS37410520,ZORLEIDOSSANTOSHARPS39410520,ZORLEIDOSSANTOSHARPS10062021
1181436,1270234,-1,PR,411695,411695,ZULMIRA APARECIDA BASSANEZI CHIELE,F,48,"COVID-19, Biologia Molecular",2021-06-17,2021-06-18,2021-06-12,3,NaT,NaT,ZULMIRAAPARECIDABASSANEZICHIELE48411695,ZULMIRAAPARECIDABASSANEZICHIELE47411695,ZULMIRAAPARECIDABASSANEZICHIELE49411695,ZULMIRAAPARECIDABASSANEZICHIELE48411695,ZULMIRAAPARECIDABASSANEZICHIELE47411695,ZULMIRAAPARECIDABASSANEZICHIELE49411695,ZULMIRAAPARECIDABASSANEZICHIELE17062021
1181437,1272700,-1,PR,412625,411520,ZULMIRA RIBEIRO MARTINS,F,46,Teste Rápido Antígeno (Imunocromatografia),2021-06-17,2021-06-18,2021-06-11,3,NaT,NaT,ZULMIRARIBEIROMARTINS46412625,ZULMIRARIBEIROMARTINS45412625,ZULMIRARIBEIROMARTINS47412625,ZULMIRARIBEIROMARTINS46411520,ZULMIRARIBEIROMARTINS45411520,ZULMIRARIBEIROMARTINS47411520,ZULMIRARIBEIROMARTINS17062021


In [26]:
cc = CasosConfirmados()
cc.df = casos_confirmados
cc.save('cc_18_06_2021',replace=True)

databases: ['cc_13_06_2021', 'cc_14_06_2021', 'cc_15_06_2021', 'cc_17_06_2021', 'cc_18_06_2021', 'cc_21_05_2021', 'cc_22_05_2021', 'cc_23_05_2021', 'cc_25_05_2021', 'cc_27_05_2021', 'cc_28_05_2021', 'cc_29_05_2021']
Load /home/vinnylg/projects/covid19datascience/bulletin/database/casos_confirmados/cc_18_06_2021.pkl
saving Casos Confirmados to pkl time elepsed:00:00:05
